In [52]:
import pandas as pd
import geopandas as gpd
import requests as r
import json

In [53]:
test_request = {
  "workforce_type": "all",
  "specialities": {'0': 0.9, '1': 1, '4': 0.1, '8': 0.5, '10': 0.9, '11': 0.2, '14': 1, '16': 1, '17': 0.6, '18': 0.5, '22': 1},
  "edu_groups": {
    "3": 0.1,
    "5": 0.5,
    "12": 0.7,
    "20": 0.5,
    "21": 0.6
    
  }
}

ontology = pd.read_csv("/var/essdata/IDU/other/mm_22/industrial-location/api//app/data/ontology.csv", index_col=0)
graduates = pd.read_csv("/var/essdata/IDU/other/mm_22/industrial-location/api/app/data/graduates.csv", index_col=0)
vacancy = pd.read_parquet("/var/essdata/IDU/other/mm_22/industrial-location/api/app/data/vacancy.gzip")
responses = pd.read_parquet("/var/essdata/IDU/other/mm_22/industrial-location/api/app/data/responses.gzip")
cv = pd.read_parquet("/var/essdata/IDU/other/mm_22/industrial-location/api/app/data/cv.gzip")
cities = gpd.read_file("/var/essdata/IDU/other/mm_22/industrial-location/api/app/data/cities.geojson")

In [54]:
# responses = r.post('http://10.32.1.62:5500/api/calculation/estimates', json=test_request)
responses = r.post('http://10.32.1.60:5000/api/calculation/estimates', json=test_request)

In [55]:
estimates = gpd.GeoDataFrame.from_features(responses.json()['estimates'])
links = gpd.GeoDataFrame.from_features(responses.json()['links'])

In [56]:
estimates[['region_city', "estimate", "num_in_migration"]]

,region_city,estimate,num_in_migration
0,"Москва, Москва",1.000,6618.0
1,"Санкт-Петербург, Санкт-Петербург",0.972,1366.0
2,"Омская область, Омск",0.963,1177.0
3,"Ростовская область, Ростов-на-Дону",0.958,1233.0
4,"Красноярский край, Красноярск",0.956,1292.0
...,...,...,...
1112,"Курская область, Дмитриев-Льговский",0.000,0.0
1113,"Московская область, Красноармейск",0.000,0.0
1114,"Московская область, Озёры",0.000,0.0
1115,"Московская область, Белоозёрский",0.000,0.0


In [57]:
test_request = {
  "city": "Пермский край, Пермь"
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/connection', json=test_request)
responses = r.post('http://10.32.1.60:5000/api/calculation/connection', json=test_request)

In [58]:
migration_link = gpd.GeoDataFrame.from_features(responses.json()['migration_link'])

In [59]:
test_request = {
  "city_name": "Пермский край, Пермь",
  'estimates_table': json.loads(estimates.to_json())
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/prediction', json=test_request)
responses = r.post('http://10.32.1.60:5000/api/calculation/prediction', json=test_request)

In [60]:
responses.json().keys()

dict_keys(['city_features', 'new_links'])

In [62]:
city_features = responses.json()['city_features']
gpd.GeoDataFrame.from_features(city_features)[["estimate_after", "estimate_before"]]

,estimate_after,estimate_before
0,0.947,0.947


In [63]:
new_links = gpd.GeoDataFrame.from_features(responses.json()['new_links'])

In [64]:
new_links.set_index("cluster_center_cv").equals(migration_link.set_index("cluster_center_cv").loc[new_links["cluster_center_cv"]])

True

In [65]:
slice = estimates["region_city"] == "Пермский край, Пермь"

In [66]:
estimates.columns

Index(['geometry', 'city', 'city_category', 'cv_count_weighted_sum',
       'cvs_count_all', 'estimate', 'factories_total',
       'graduates_forecast_number', 'graduates_forecast_sum_number',
       'graduates_weighted_sum', 'harsh_climate', 'index', 'max_salary',
       'median_salary', 'min_salary', 'num_in_migration', 'num_out_migration',
       'num_responses', 'one_vacancy_out_response', 'population',
       'probability_to_move', 'region', 'region_city', 'specialists_number',
       'specialists_sum_number', 'ueqi_citywide_space', 'ueqi_green_spaces',
       'ueqi_public_and_business_infrastructure', 'ueqi_residential',
       'ueqi_score', 'ueqi_social_and_leisure_infrastructure',
       'ueqi_street_networks', 'vacancy_count'],
      dtype='object')

In [67]:
estimates.loc[slice, 'ueqi_social_and_leisure_infrastructure'] += 25
estimates.loc[slice, 'ueqi_public_and_business_infrastructure'] += 25
estimates.loc[slice, 'ueqi_citywide_space'] += 25

In [70]:
test_request = {
  "city_name": "Пермский край, Пермь",
  'estimates_table': json.loads(estimates.to_json())
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/prediction', json=test_request)
responses = r.post('http://10.32.1.60:5000/api/calculation/prediction', json=test_request)

In [74]:
city_features = responses.json()['city_features']
gpd.GeoDataFrame.from_features(city_features)[["estimate_after", "estimate_before", "num_in_migration_before", "num_in_migration_after"]]

,estimate_after,estimate_before,num_in_migration_before,num_in_migration_after
0,0.982,0.947,718.0,3383.0


In [73]:
gpd.GeoDataFrame.from_features(city_features).columns

Index(['geometry', 'city', 'city_category', 'cv_count_weighted_sum',
       'cvs_count_all', 'estimate_after', 'estimate_before', 'factories_total',
       'graduates_forecast_number', 'graduates_forecast_sum_number',
       'graduates_weighted_sum', 'harsh_climate', 'index', 'max_salary',
       'median_salary', 'min_salary', 'num_in_migration_after',
       'num_in_migration_before', 'num_out_migration', 'num_responses',
       'one_vacancy_out_response', 'population', 'probability_to_move',
       'region', 'specialists_number', 'specialists_sum_number',
       'ueqi_citywide_space', 'ueqi_green_spaces',
       'ueqi_public_and_business_infrastructure', 'ueqi_residential',
       'ueqi_score', 'ueqi_social_and_leisure_infrastructure',
       'ueqi_street_networks', 'vacancy_count'],
      dtype='object')

In [46]:
responses.json()["update_dict"]

{'before': {'estimate': 0.947, 'in_migration': 718},
 'after': {'estimate': 0.982, 'in_migration': 3383}}